# Interactive Map - Folium (Leaflet)

For experimentation on creating an interactive map with our data.

In [1]:
### Import Libraries

# File manipulation

import os # For working with Operating System
from sys import platform # Diagnose operating system
import json # For working with Json files
import requests # Processing online requests


# Analysis

import numpy as np # For working with Arrays
import pandas as pd # Data Manipulation
import geopandas as gpd # Spatial Data Manipulation

# Visualization

import folium # Interactive Leaflet.js mapping 
from folium import features

import warnings
warnings.filterwarnings('ignore') # Ignores some warnings

In [2]:
### Definitions

cwd = os.getcwd() # Current Working Directory

# Forward or back slashs for filepaths? <- Not sure here. Only know Windows & Linux

if platform == "linux" or platform == "linux2":
    slash = '/'
elif platform == 'win32':
    slash = '\\'

# Load Data

In [3]:
# Load Data

# Get GeoJsons from GitHub

# Base URL

url = (
    "https://raw.githubusercontent.com/RwHendrickson/MappingGZ/main/Prototype/Notebooks/CleaningData"
)

# Minneapolis Boundary

mpls_boundary_path = f"{url}/Boundary/mpls_boundaryWGS84.geojson"

# GZ Boundaries

gz_boundary_path = f"{url}/GZ_Boundaries/gz_boundaryWGS84.geojson"

# Get bigger files from computer

cwd = os.getcwd()

mpls_aadt_path = cwd + slash + 'CleaningData' + slash + 'Traffic' + slash + 'mpls_aadtWGS84.geojson'
mpls_emissions_path = cwd + slash + 'CleaningData' + slash + 'PermittedEmissions' + slash + 'mpls_emissions.csv'

# Load files as geodataframes for focusing data

traffic = gpd.read_file(mpls_aadt_path)
emissions = pd.read_csv(mpls_emissions_path)

emissions_geo = gpd.GeoDataFrame(emissions, geometry = gpd.points_from_xy(emissions.LONGITUDE, emissions.LATITUDE, crs = 'EPSG:4326'))

In [4]:
# Focus Data

## Traffic

aadt = traffic[['SEQUENCE_N', 'ROUTE_LABE', 'CURRENT_VO', 'geometry']]
big_aadt = aadt[aadt['CURRENT_VO'] > 10000]

# Emissions

emissions_2020 = emissions_geo[emissions_geo['YEAR']==2020][['FACILITY_NAME', 'INDUSTRY_TYPE', 'NAICS_CODE', 'POLLUTANT', 'LBS_EMITTED','geometry']]
big_vocs = emissions_2020[(emissions_2020['POLLUTANT'] == 'Volatile Organic Compounds') & 
                         (emissions_2020['LBS_EMITTED'] > 1000)]
big_pm = emissions_2020[(emissions_2020['POLLUTANT'] == 'PM2.5 Primary') & 
                         (emissions_2020['LBS_EMITTED'] > 1000)]

# Define Visualizations

## Annual Average Daily Traffic

In [5]:
### AADT style

def style_aadt(feature):
    '''Set Thickness/color of streets to be proportional to traffic volume'''
    
    volume = feature['properties']['CURRENT_VO']
    
    if volume < 1773: # Low volume
        return {
        "weight": 0.5,
        "color": "#848484"
    }
    elif volume < 5378: # Low-mid volume
        return {
        "weight": 1,
        "color": "#936d6d"
    }
    elif volume < 16308: # Mid volume
        return {
        "weight": 2,
        "color": "#a94646"
    }
    elif volume < 100000: # High Volume
        return {
        "weight": 5,
        "color": "#f90707"
    }
    else: # Very High Volume Potentially could use another break here
        return {
        "weight": 10,
        "color": "#090707"
    }
    

## Permitted Emissions

### Popups

In [6]:
def emissions_popup_html(point):
    '''Creates a popup with basic info on an industrial polluter'''

    left_col_color = "#627c6a"
    right_col_color = "#bccdb9"
    
    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(point.FACILITY_NAME) + """

</head>
    <table style="height: 100px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Pollutant:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.POLLUTANT) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Pounds Emitted (2020):</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.LBS_EMITTED) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">Industry Type:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.INDUSTRY_TYPE) + """
</tr>
<tr>
<td style="background-color: """+ left_col_color +""";"><span style="color: #ffffff;">NAICS code:</span></td>
<td style="width: 150px;background-color: """+ right_col_color +""";">{}</td>""".format(point.NAICS_CODE) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

### VOC Emissions

In [7]:
# Plot the facilities' VOC emission
    
def plot_voc(point):
    '''plot circle of facilities related to amount of vocs emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    popup = emissions_popup_html(point)
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       fill = True,
                       popup = popup
                       ).add_to(voc_feature_group)

## PM2.5 Emissions

In [8]:
# Plot the facilities' VOC emission
    
def plot_pm(point):
    '''plot circle of facilities related to amount of particulate matter 2.5 emitted'''    
    
    lbs = point.LBS_EMITTED 
    
    if lbs < 25: # Low volume
        weight = 1,
        color = '#848484'
    elif lbs < 100: # Low-mid volume
        weight = 3,
        color = '#936d6d'
    elif lbs < 1000: # Mid volume
        weight = 5,
        color = '#a94646'
    elif lbs < 10000: # High Volume
        weight = 10,
        color = '#f90707'
    else: # Very High Volume
        weight = 15,
        color = '#090707'
        
    popup = emissions_popup_html(point)
        
    folium.Circle(location=[point.geometry.y,point.geometry.x],
                       weight = weight,
                       color = color,
                       fill = True,
                       popup = popup
                       ).add_to(pm_feature_group)

# Add to the map!

In [9]:
# Make a basic folium map

m = folium.Map(location=[44.986656, -93.258133],
               zoom_start=12,
               tiles = 'cartodbpositron')

# Add boundary

folium.GeoJson(mpls_boundary_path,
               style_function = lambda feature: {'color':'#000000', # Border color
                                                'fillOpacity' : '0'}, # Fill transparency
               name="Minneapolis Boundary", # Layer's Name
               control = False # Can you toggle in layer control?
              ).add_to(m)

# Add GZ boundaries

folium.GeoJson(gz_boundary_path, 
               style_function = lambda feature: {'fillColor':'#84E884',
                                 'color':'#003700',
                                 'fillOpacity': '0.5',
                                 'weight': '0'}, # Thickness of lines
               name = "Green Zone Boundaries",
               control = False).add_to(m)

# Add traffic

folium.GeoJson(big_aadt,
               style_function = lambda feature: style_aadt(feature), 
               name = 'Annual Average Daily Traffic',
               show = False # Does this appear in default of map
              ).add_to(m)

# Add Volatile Organic Compounds

voc_feature_group = folium.FeatureGroup(name = 'VOC Emitters',
                                           show = False) # Create Feature Group
big_vocs.apply(plot_voc, axis = 1) # Add to group
m.add_child(voc_feature_group)

# Add PM2.5

pm_feature_group = folium.FeatureGroup(name = 'PM2.5 Emitters',
                                           show = False) # Create Feature Group
big_pm.apply(plot_pm, axis = 1) # Add to group
m.add_child(pm_feature_group)


# Turn on layer control

m.add_child(folium.map.LayerControl())

In [10]:
# Save Map

m.save(os.path.join('.', 'MVP.html'))